In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.preprocessing import preprocess_input_df

/Users/lorenh/Documents/DSaPP/.aequitas/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv("../../../examples/data/compas_for_aequitas.csv")
df.head()

,entity_id,score,label_value,race,sex,age_cat
0,1,0.0,0,Other,Male,Greater than 45
1,3,0.0,1,African-American,Male,25 - 45
2,4,0.0,1,African-American,Male,Less than 25
3,5,1.0,0,African-American,Male,Less than 25
4,6,0.0,0,Other,Male,25 - 45


In [3]:
df.shape

(7214, 6)

## Pre-Aequitas: Exploring the COMPAS Dataset

__Risk assessment by race__

COMPAS produces a risk score that predicts a person's likelihood of commiting a crime in the next two years. The output is a score between 1 and 10 that maps to low, medium or high. For Aequitas, we collapse this to a binary prediction. A score of 0 indicates a prediction of "low" risk according to COMPAS, while a 1 indicates "high" or "medium" risk.

This categorization is based on ProPublica's interpretation of Northpointe's practioner guide:

    "According to Northpointe’s practitioners guide, COMPAS “scores in the medium and high range garner more interest from supervision agencies than low scores, as a low score would suggest there is little risk of general recidivism,” so we considered scores any higher than “low” to indicate a risk of recidivism."

In the bar charts below, we see a large difference in how these scores are distributed by race, with a majority of white and Hispanic people predicted as low risk (score = 0) and a majority of black people predicted high and medium risk (score = 1). We also see that while the majority of people in age categories over 25 are predicted as low risk (score = 0), the majority of people below 25 are predicted as high and medium risk (score = 1).

### Data Formatting

Data for this example was preprocessed for compatibility with Aequitas. **The Aequitas tool always requires a `score` column and requires a binary `label_value` column for supervised metrics**, (i.e., False Discovery Rate, False Positive Rate, False Omission Rate, and False Negative Rate).

Preprocessing includes but is not limited to checking for mandatory `score` and `label_value` columns as well as at least one column representing attributes specific to the data set. See [documentation](../input_data.html) for more information about input data.

Note that while `entity_id` is not necessary for this example, Aequitas recognizes `entity_id` as a reserve column name and will not recognize it as an attribute column.

[Back to Top](#top_cell)
<a id='existing_biases'></a>
## What biases exist in my model?
### Aequitas Group() Class

<a id='xtab'></a>
### What is the distribution of groups, predicted scores, and labels across my dataset?

Aequitas's `Group()` class enables researchers to evaluate biases across all subgroups in their dataset by assembling a confusion matrix of each subgroup, calculating commonly used metrics such as false positive rate and false omission rate, as well as counts by group and group prevelance among the sample population. 

The **`get_crosstabs()`** command tabulates a confusion matrix for each subgroup and calculates commonly used metrics such as false positive rate and false omission rate. It also provides counts by group and group prevelances.

#### Group Counts Calculated:

| Count Type | Column Name |
| --- | --- |
| False Positive Count | 'fp' |
| False Negative Count | 'fn' |
| True Negative Count | 'tn' |
| True Positive Count | 'tp' |
| Predicted Positive Count | 'pp' |
| Predicted Negative Count | 'pn' |
| Count of Negative Labels in Group | 'group_label_neg' |
| Count of Positive Labels in Group | 'group_label_pos' | 
| Group Size | 'group_size'|
| Total Entities | 'total_entities' |

#### Absolute Metrics Calcuated:

| Metric | Column Name |
| --- | --- |
| True Positive Rate | 'tpr' |
| True Negative Rate | 'tnr' |
| False Omission Rate | 'for' |
| False Discovery Rate | 'fdr' |
| False Positive Rate | 'fpr' |
| False Negative Rate | 'fnr' |
| Negative Predictive Value | 'npv' |
| Precision | 'precision' |
| Predicted Positive Ratio$_k$ | 'ppr' |
| Predicted Positive Ratio$_g$ | 'pprev' |
| Group Prevalence | 'prev' |


**Note**: The **`get_crosstabs()`** method expects a dataframe with predefined columns `score`, and `label_value` and treats other columns (with a few exceptions) as attributes against which to test for disparities. In this cases we include `race`, `sex` and `age_cat`. 

In [4]:
g = Group()
xtab, _, score_thresholds_dict = g.get_crosstabs(df)

model_id, score_thresholds 1 {'rank_abs': [3317]}
COUNTS::: race
African-American    3696
Asian                 32
Caucasian           2454
Hispanic             637
Native American       18
Other                377
dtype: int64
COUNTS::: sex
Female    1395
Male      5819
dtype: int64
COUNTS::: age_cat
25 - 45            4109
Greater than 45    1576
Less than 25       1529
dtype: int64


In [5]:
preprocessed_df, _ = preprocess_input_df(df, required_cols=['score', 'label_value'])

In [6]:
preprocessed_df.head()

,entity_id,score,label_value,race,sex,age_cat
0,1,0.0,0,Other,Male,Greater than 45
1,3,0.0,1,African-American,Male,25 - 45
2,4,0.0,1,African-American,Male,Less than 25
3,5,1.0,0,African-American,Male,Less than 25
4,6,0.0,0,Other,Male,25 - 45


In [7]:
df.head()

,entity_id,score,label_value,race,sex,age_cat
0,1,0.0,0,Other,Male,Greater than 45
1,3,0.0,1,African-American,Male,25 - 45
2,4,0.0,1,African-American,Male,Less than 25
3,5,1.0,0,African-American,Male,Less than 25
4,6,0.0,0,Other,Male,25 - 45


[Back to Top](#top_cell)
<a id='xtab_metrics'></a>
### What are bias metrics across groups?

Once you have run the `Group()` class, you'll have a dataframe of the group counts and group value bias metrics.

The `Group()` class has a **`list_absolute_metrics()`** method, which you can use for faster slicing to view just  counts or bias metrics.

In [8]:
absolute_metrics = g.list_absolute_metrics(xtab)

#### View calculated counts across sample population groups

In [9]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

,attribute_name,attribute_value,k,model_id,score_threshold,pp,pn,fp,fn,tn,tp,group_label_neg,group_label_pos,group_size,total_entities
0,race,African-American,3317,1,binary 0/1,2174,1522,805,532,990,1369,1795,1901,3696,7214
1,race,Asian,3317,1,binary 0/1,8,24,2,3,21,6,23,9,32,7214
2,race,Caucasian,3317,1,binary 0/1,854,1600,349,461,1139,505,1488,966,2454,7214
3,race,Hispanic,3317,1,binary 0/1,190,447,87,129,318,103,405,232,637,7214
4,race,Native American,3317,1,binary 0/1,12,6,3,1,5,9,8,10,18,7214
5,race,Other,3317,1,binary 0/1,79,298,36,90,208,43,244,133,377,7214
6,sex,Female,3317,1,binary 0/1,591,804,288,195,609,303,897,498,1395,7214
7,sex,Male,3317,1,binary 0/1,2726,3093,994,1021,2072,1732,3066,2753,5819,7214
8,age_cat,25 - 45,3317,1,binary 0/1,1924,2185,741,706,1479,1183,2220,1889,4109,7214
9,age_cat,Greater than 45,3317,1,binary 0/1,394,1182,181,285,897,213,1078,498,1576,7214


#### View calculated absolute metrics for each sample population group

In [10]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2)

,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,race,African-American,0.72,0.55,0.35,0.37,0.45,0.28,0.65,0.63,0.66,0.59,0.51
1,race,Asian,0.67,0.91,0.12,0.25,0.09,0.33,0.88,0.75,0.00,0.25,0.28
2,race,Caucasian,0.52,0.77,0.29,0.41,0.23,0.48,0.71,0.59,0.26,0.35,0.39
3,race,Hispanic,0.44,0.79,0.29,0.46,0.21,0.56,0.71,0.54,0.06,0.30,0.36
4,race,Native American,0.90,0.62,0.17,0.25,0.38,0.10,0.83,0.75,0.00,0.67,0.56
5,race,Other,0.32,0.85,0.30,0.46,0.15,0.68,0.70,0.54,0.02,0.21,0.35
6,sex,Female,0.61,0.68,0.24,0.49,0.32,0.39,0.76,0.51,0.18,0.42,0.36
7,sex,Male,0.63,0.68,0.33,0.36,0.32,0.37,0.67,0.64,0.82,0.47,0.47
8,age_cat,25 - 45,0.63,0.67,0.32,0.39,0.33,0.37,0.68,0.61,0.58,0.47,0.46
9,age_cat,Greater than 45,0.43,0.83,0.24,0.46,0.17,0.57,0.76,0.54,0.12,0.25,0.32


In [11]:
b = Bias()

In [21]:
rd = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict={'sex': 'Female' , 'race': 'Hispanic', 'age_cat': '25 - 45'}, mask_significance=True)
rd

get_disparity_predefined_group()


,attribute_name,attribute_value,k,model_id,score_threshold,tpr,tnr,for,fdr,fpr,...,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value,fn_siginificance,fp_siginificance,score_siginificance,label_value_siginificance
0,race,African-American,3317,1,binary 0/1,0.720147,0.551532,0.349540,0.370285,0.448468,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,False,True,False,True
1,race,Asian,3317,1,binary 0/1,0.666667,0.913043,0.125000,0.250000,0.086957,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,True,False
2,race,Caucasian,3317,1,binary 0/1,0.522774,0.765457,0.288125,0.408665,0.234543,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,False,False
3,race,Hispanic,3317,1,binary 0/1,0.443966,0.785185,0.288591,0.457895,0.214815,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,True,False
4,race,Native American,3317,1,binary 0/1,0.900000,0.625000,0.166667,0.250000,0.375000,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,False,False
5,race,Other,3317,1,binary 0/1,0.323308,0.852459,0.302013,0.455696,0.147541,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,False,False
6,sex,Female,3317,1,binary 0/1,0.608434,0.678930,0.242537,0.487310,0.321070,...,Female,Female,Female,Female,Female,Female,True,False,True,False
7,sex,Male,3317,1,binary 0/1,0.629132,0.675799,0.330100,0.364637,0.324201,...,Female,Female,Female,Female,Female,Female,False,True,False,True
8,age_cat,25 - 45,3317,1,binary 0/1,0.626257,0.666216,0.323112,0.385135,0.333784,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,True,False,True,False
9,age_cat,Greater than 45,3317,1,binary 0/1,0.427711,0.832096,0.241117,0.459391,0.167904,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,True,True,False,True


In [13]:
5e-2

0.05

In [14]:
rd

,attribute_name,attribute_value,k,model_id,score_threshold,tpr,tnr,for,fdr,fpr,...,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value,fn_siginificance,fp_siginificance,score_siginificance,label_value_siginificance
0,race,African-American,3317,1,binary 0/1,0.720147,0.551532,0.349540,0.370285,0.448468,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,False,True,False,True
1,race,Asian,3317,1,binary 0/1,0.666667,0.913043,0.125000,0.250000,0.086957,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,True,False
2,race,Caucasian,3317,1,binary 0/1,0.522774,0.765457,0.288125,0.408665,0.234543,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,False,False
3,race,Hispanic,3317,1,binary 0/1,0.443966,0.785185,0.288591,0.457895,0.214815,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,True,False
4,race,Native American,3317,1,binary 0/1,0.900000,0.625000,0.166667,0.250000,0.375000,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,False,False
5,race,Other,3317,1,binary 0/1,0.323308,0.852459,0.302013,0.455696,0.147541,...,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,Hispanic,True,False,False,False
6,sex,Female,3317,1,binary 0/1,0.608434,0.678930,0.242537,0.487310,0.321070,...,Female,Female,Female,Female,Female,Female,True,False,True,False
7,sex,Male,3317,1,binary 0/1,0.629132,0.675799,0.330100,0.364637,0.324201,...,Female,Female,Female,Female,Female,Female,False,True,False,True
8,age_cat,25 - 45,3317,1,binary 0/1,0.626257,0.666216,0.323112,0.385135,0.333784,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,True,False,True,False
9,age_cat,Greater than 45,3317,1,binary 0/1,0.427711,0.832096,0.241117,0.459391,0.167904,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,True,True,False,True


In [15]:
bodf = b.get_disparity_major_group(xtab)

get_disparity_major_group()


In [16]:
default_key_columns = ['model_id', 'score_threshold', 'attribute_name']

In [17]:
df_min_idx = bodf.loc[bodf.groupby(default_key_columns)['fpr'].idxmin()]
df_min_idx

,attribute_name,attribute_value,k,model_id,score_threshold,tpr,tnr,for,fdr,fpr,...,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
9,age_cat,Greater than 45,3317,1,binary 0/1,0.427711,0.832096,0.241117,0.459391,0.167904,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45
1,race,Asian,3317,1,binary 0/1,0.666667,0.913043,0.125000,0.250000,0.086957,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
6,sex,Female,3317,1,binary 0/1,0.608434,0.678930,0.242537,0.487310,0.321070,...,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male


In [18]:
df_min_idx.loc[df_min_idx['attribute_name'] == 'race']

,attribute_name,attribute_value,k,model_id,score_threshold,tpr,tnr,for,fdr,fpr,...,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
1,race,Asian,3317,1,binary 0/1,0.666667,0.913043,0.125,0.25,0.086957,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American


In [19]:
b.get_disparity_min_metric(xtab)

get_disparity_min_metric()
get_disparity_min_metric


,attribute_name,attribute_value,k,model_id,score_threshold,tpr,tnr,for,fdr,fpr,...,fpr_disparity,fpr_ref_group_value,fnr_disparity,fnr_ref_group_value,tpr_disparity,tpr_ref_group_value,tnr_disparity,tnr_ref_group_value,npv_disparity,npv_ref_group_value
0,race,African-American,3317,1,binary 0/1,0.720147,0.551532,0.349540,0.370285,0.448468,...,5.157382,Asian,2.798527,Native American,2.227432,Other,1.000000,African-American,1.000000,African-American
1,race,Asian,3317,1,binary 0/1,0.666667,0.913043,0.125000,0.250000,0.086957,...,1.000000,Asian,3.333333,Native American,2.062016,Other,1.655468,African-American,1.345202,African-American
2,race,Caucasian,3317,1,binary 0/1,0.522774,0.765457,0.288125,0.408665,0.234543,...,2.697245,Asian,4.772257,Native American,1.616953,Other,1.387874,African-American,1.094418,African-American
3,race,Hispanic,3317,1,binary 0/1,0.443966,0.785185,0.288591,0.457895,0.214815,...,2.470370,Asian,5.560345,Native American,1.373196,Other,1.423644,African-American,1.093702,African-American
4,race,Native American,3317,1,binary 0/1,0.900000,0.625000,0.166667,0.250000,0.375000,...,4.312500,Asian,1.000000,Native American,2.783721,Other,1.133207,African-American,1.281145,African-American
5,race,Other,3317,1,binary 0/1,0.323308,0.852459,0.302013,0.455696,0.147541,...,1.696721,Asian,6.766917,Native American,1.000000,Other,1.545620,African-American,1.073066,African-American
6,sex,Female,3317,1,binary 0/1,0.608434,0.678930,0.242537,0.487310,0.321070,...,1.000000,Female,1.055810,Male,1.000000,Female,1.004633,Male,1.130710,Male
7,sex,Male,3317,1,binary 0/1,0.629132,0.675799,0.330100,0.364637,0.324201,...,1.009751,Female,1.000000,Male,1.034019,Female,1.000000,Male,1.000000,Male
8,age_cat,25 - 45,3317,1,binary 0/1,0.626257,0.666216,0.323112,0.385135,0.333784,...,1.987950,Greater than 45,1.435172,Less than 25,1.464207,Greater than 45,1.452570,Less than 25,1.176231,Less than 25
9,age_cat,Greater than 45,3317,1,binary 0/1,0.427711,0.832096,0.241117,0.459391,0.167904,...,1.000000,Greater than 45,2.197590,Less than 25,1.000000,Greater than 45,1.814243,Less than 25,1.318715,Less than 25
